## Deliverable 2. Create a Customer Travel Destinations Map.

In [43]:
# Dependencies
%matplotlib inline

import matplotlib.pyplot as plt
import json
import requests
import os
import pandas as pd
import numpy as np
import gmaps
import requests

from scipy.stats import linregress
from dotenv import load_dotenv
from citipy import citipy
from datetime import datetime
import time
load_dotenv()
api_key = os.getenv("OpenWeather_API_KEY")
g_key = os.getenv("g_key")

In [44]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description
0,0,Aguas Vermelhas,-15.7472,-41.4600,56.28,98,21,2.39,BR,2022-07-20 05:34:22,few clouds
1,1,Adrar,20.5022,-10.0711,85.60,17,15,5.39,MR,2022-07-20 05:34:23,few clouds
2,2,Albany,42.6001,-73.9662,72.54,87,51,4.88,US,2022-07-20 05:32:08,broken clouds
3,3,Mataura,-46.1927,168.8643,43.48,84,97,2.39,NZ,2022-07-20 05:34:23,overcast clouds
4,4,Tiksi,71.6872,128.8694,34.14,95,100,13.06,RU,2022-07-20 05:34:24,overcast clouds


In [45]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 80


In [47]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description
6,6,Osypenko,46.9136,36.8253,76.48,33,3,4.12,UA,2022-07-20 05:34:25,clear sky
19,19,Surt,31.2089,16.5887,78.62,67,0,4.09,LY,2022-07-20 05:34:29,clear sky
27,27,Carnarvon,-24.8667,113.6333,76.87,46,61,15.59,AU,2022-07-20 05:34:32,broken clouds
55,55,Alofi,-19.0595,-169.9187,75.09,78,10,16.11,NU,2022-07-20 05:35:41,clear sky
65,65,Mutis,6.2262,-77.4044,77.97,89,100,5.32,CO,2022-07-20 05:35:45,moderate rain
72,72,Ambilobe,-13.2000,49.0500,78.75,40,2,12.95,MG,2022-07-20 05:35:48,clear sky
76,76,Miyang,24.4042,103.4428,77.68,81,99,5.08,CN,2022-07-20 05:35:49,moderate rain
86,86,Daru,-9.0763,143.2092,78.24,88,100,12.46,PG,2022-07-20 05:35:53,moderate rain
100,100,Bethel,41.3712,-73.4140,78.55,82,0,0.00,US,2022-07-20 05:36:58,clear sky
102,102,Noumea,-22.2763,166.4572,77.00,99,99,1.43,NC,2022-07-20 05:36:59,overcast clouds


In [48]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isna().sum().sum()

len(preferred_cities_df)

112

In [49]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [50]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
6,Osypenko,UA,76.48,clear sky,46.9136,36.8253,
19,Surt,LY,78.62,clear sky,31.2089,16.5887,
27,Carnarvon,AU,76.87,broken clouds,-24.8667,113.6333,
55,Alofi,NU,75.09,clear sky,-19.0595,-169.9187,
65,Mutis,CO,77.97,moderate rain,6.2262,-77.4044,
72,Ambilobe,MG,78.75,clear sky,-13.2000,49.0500,
76,Miyang,CN,77.68,moderate rain,24.4042,103.4428,
86,Daru,PG,78.24,moderate rain,-9.0763,143.2092,
100,Bethel,US,78.55,clear sky,41.3712,-73.4140,
102,Noumea,NC,77.00,overcast clouds,-22.2763,166.4572,


In [76]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

record_count = 1 

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # Adding ouput counter
    
    print(f"Processing Record {record_count}")
    record_count += 1
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Processing Record 1
Hotel not found... skipping.
Processing Record 2
Processing Record 3
Processing Record 4
Processing Record 5
Processing Record 6
Processing Record 7
Processing Record 8
Processing Record 9
Processing Record 10
Processing Record 11
Processing Record 12
Hotel not found... skipping.
Processing Record 13
Processing Record 14
Processing Record 15
Processing Record 16
Processing Record 17
Processing Record 18
Processing Record 19
Processing Record 20
Processing Record 21
Processing Record 22
Processing Record 23
Processing Record 24
Processing Record 25
Hotel not found... skipping.
Processing Record 26
Processing Record 27
Processing Record 28
Processing Record 29
Hotel not found... skipping.
Processing Record 30
Processing Record 31
Processing Record 32
Processing Record 33
Processing Record 34
Processing Record 35
Processing Record 36
Hotel not found... skipping.
Processing Record 37
Hotel not found... skipping.
Processing Record 38
Processing Record 39
Processing Recor

In [77]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != ""]
clean_hotel_df.set_index("City")

,Country,Max Temp,Description,Lat,Lng,Hotel Name
City,,,,,,
Surt,LY,78.62,clear sky,31.2089,16.5887,الهادي ابوشعاله سرت
Carnarvon,AU,76.87,broken clouds,-24.8667,113.6333,Hospitality Carnarvon
Alofi,NU,75.09,clear sky,-19.0595,-169.9187,Taloa Heights
Mutis,CO,77.97,moderate rain,6.2262,-77.4044,Paraiso Escondido
Ambilobe,MG,78.75,clear sky,-13.2000,49.0500,Hôtel & Restaurant DIANA Ambilobe
...,...,...,...,...,...,...
Fernley,US,78.75,scattered clouds,39.6080,-119.2518,Super 8 by Wyndham Fernley
Saint Anthony,US,75.92,few clouds,45.0205,-93.2180,"Holiday Inn Express Roseville-St. Paul, an IHG..."
Coahuayana,MX,78.39,broken clouds,18.7333,-103.6833,El rranchito michuacan de ocampos


In [72]:
# 8a. Create the output File (CSV)
# Create the output file (CSV).
output_data_file = "WeatherPy_vacation.csv"


# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [60]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Description</dt><dd>{Description}</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [61]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))